# Migration simulation in EU

### Multipliers
Each element included in happiness is multiply by corresponding multiplier. At the end all elements are summed up.

In [1]:
MULTIPLIER_INTEGRATION_WITH_IMMIGRANTS = 0.1

MULTIPLIER_RACE_1ST = 0.3
MULTIPLIER_RACE_2ND = 0.2
MULTIPLIER_RACE_3RD = 0.1

MULTIPLIER_RELIGIOUS_DENOMINATION_1ST = 0.3
MULTIPLIER_RELIGIOUS_DENOMINATION_2ND = 0.2
MULTIPLIER_RELIGIOUS_DENOMINATION_3RD = 0.1

MULTIPLIER_SENSE_OF_COMFORT = 0.1

MULTIPLIER_MIGRATION_INTO_OUR_COUNTRY = 0.1

### Neighboring countries
Agent can migrate only to neighboring countries.

In [2]:
NEIGHBORING_COUNTRIES = {
    'Austria': ['Germany', 'Italy', 'Slovenia', 'Hungary', 'Slovakia', 'Czech Republic'],
    'Belgium': ['France', 'Germany', 'Luxemburg', 'Netherlands', 'Ireland', 'United Kingdom'],
    'Bulgaria': ['Greece', 'Romania'],
    'Croatia': ['Hungary', 'Slovenia'],
    'Cyprus': ['Greece'],
    'Czech Republic': ['Slovakia', 'Poland', 'Germany', 'Austria'],
    'Denmark': ['Germany', 'Sweden', 'United Kingdom'],
    'Estonia': ['Sweden', 'Finland', 'Latvia'],
    'Finland': ['Sweden', 'Estonia'],
    'France': ['Belgium', 'Luxemburg', 'Germany', 'Italy', 'Spain', 'United Kingdom', 'Netherlands'],
    'Greece': ['Bulgaria','Cyprus', 'Italy'],
    'Spain': ['Portugal', 'France'],
    'Ireland': ['United Kingdom', 'Netherlands', 'Belgium'],
    'Lithuania': ['Latvia', 'Poland', 'Sweden'],
    'Luxemburg': ['Germany', 'Belgium', 'France'],
    'Latvia': ['Estonia', 'Lithuania', 'Sweden'],
    'Malta': ['Italy'],
    'Netherlands': ['Germany', 'Belgium', 'France', 'United Kingdom', 'Ireland'],
    'Germany': ['Denmark', 'Poland', 'Czech Republic', 'Austria', 'France', 'Luxemburg', 'Belgium', 'Netherlands', 'Sweden', 'United Kingdom'],
    'Poland': ['Germany', 'Lithuania', 'Slovakia', 'Czech Republic', 'Sweden'],
    'Portugal': ['Spain'],
    'Romania': ['Bulgaria', 'Hungary'],
    'Slovakia': ['Czech Republic', 'Austria', 'Hungary', 'Poland'],
    'Slovenia': ['Austria', 'Croatia', 'Hungary', 'Italy'],
    'Sweden': ['Denmark', 'Germany', 'Poland', 'Lithuania', 'Latvia', 'Estonia'],
    'Hungary': ['Slovakia', 'Romania', 'Croatia', 'Slovenia', 'Austria'],
    'United Kingdom': ['Belgium', 'Denmark', 'France', 'Germany', 'Netherlands', 'Ireland'],
    'Italy': ['Austria', 'France', 'Slovenia', 'Greece', 'Malta']
}

Choosing next country is based on probability which depends on happiness values.
Sometimes happiness can be lower than 0, so I decided to use sigmoid function to change them to values from [0-1] interval.

In [3]:
import numpy as np


def sigmoid(n):
    return 1 / (1 + np.exp(-n))

In [4]:
from mesa import Agent


class Person(Agent):

    def __init__(self, model, unique_id, origin_country, current_country, race, religious_denomination,
                 likely_to_travel, conditions, neigboring_countries=NEIGHBORING_COUNTRIES):
        super().__init__(unique_id, model)
        self.origin_country = origin_country
        self.current_country = current_country
        self.race = race
        self.religious_denomination = religious_denomination
        self.likely_to_travel = likely_to_travel
        self.conditions = conditions
        self.happiness = self._calculate_happiness(self.current_country)
        self.neigboring_countries = neigboring_countries

    def step(self):
        # TODO: change likely_to_travel to 0-1
        # TODO: add current_country with value current_happiness*likely_to_travel or 1-likely_to_travel to happiness_index
        # TODO: Then agent will be able to stay in current country.
        happiness_index = {}
        for country in self.neigboring_countries.get(self.current_country):
            happiness_country = self._calculate_happiness(country)
            happiness_index[country] = happiness_country

        happiness_probabilities = sigmoid(np.array(list(happiness_index.values())))

        next_country = np.random.choice(np.array(list(happiness_index.keys())), 1,
                                        p=happiness_probabilities / np.sum(happiness_probabilities))[0]
        self.current_country = next_country
        self.happiness = happiness_index[next_country]

    def _calculate_happiness(self, country):
        happiness = 0
        happiness += self._calculate_happines_based_on_race(country)
        happiness += self._calculate_happiness_based_on_religious_denomination(country)
        if country != self.origin_country:
            happiness += self._calculate_happiness_based_on_a_sense_of_comfort_with_immigrants(country)
            happiness += self._calculate_happiness_based_on_integration_frequency_with_immigrants(country)
            happiness += self._calculate_happiness_based_on_thoughts_about_migration_into_own_country(country)
        return happiness

    def _calculate_happines_based_on_race(self, country):
        happiness_race = {}
        beggining_of_column_name = 'Would you feel comfortable if one of your children was in a relationship with '
        happiness_race['White'] = self.conditions[beggining_of_column_name + 'a White person?'][country]
        happiness_race['Black'] = self.conditions[beggining_of_column_name + 'a Black person?'][country]
        happiness_race['Asian'] = self.conditions[beggining_of_column_name + 'an Asian person?'][country]
        happiness_race_sorted = sorted(happiness_race, key=happiness_race.get)
        idx = happiness_race_sorted.index(self.race)

        value_for_agents_race = 0
        if self.race == 'White':
            value_for_agents_race = self.conditions[beggining_of_column_name + 'a White person?'][country]
        elif self.race == 'Black':
            value_for_agents_race = self.conditions[beggining_of_column_name + 'a Black person?'][country]
        elif self.race == 'Asian':
            value_for_agents_race = self.conditions[beggining_of_column_name + 'an Asian person?'][country]

        if idx == 0:
            return MULTIPLIER_RACE_1ST * value_for_agents_race
        elif idx == 1:
            return MULTIPLIER_RACE_2ND * value_for_agents_race
        elif idx == 2:
            return MULTIPLIER_RACE_3RD * value_for_agents_race

    def _calculate_happiness_based_on_religious_denomination(self, country):
        happiness_religious_denomination = {}
        beggining_of_column_name = 'Would you feel comfortable if one of your children was in a relationship with a '
        happiness_religious_denomination['Jew'] = self.conditions[beggining_of_column_name + 'Jew?'][country]
        happiness_religious_denomination['Muslim'] = self.conditions[beggining_of_column_name + 'Muslim?'][country]
        happiness_religious_denomination['Christian'] = self.conditions[beggining_of_column_name + 'Christian?'][country]
        happiness_religious_denomination_sorted = sorted(happiness_religious_denomination,
                                                         key=happiness_religious_denomination.get)
        idx = happiness_religious_denomination_sorted.index(self.religious_denomination)

        value_for_agents_religious_denomination = 0
        if self.religious_denomination == 'Jew':
            value_for_agents_religious_denomination = self.conditions[beggining_of_column_name + 'Jew?'][country]
        elif self.religious_denomination == 'Muslim':
            value_for_agents_religious_denomination = self.conditions[beggining_of_column_name + 'Muslim?'][country]
        elif self.religious_denomination == 'Christian':
            value_for_agents_religious_denomination = self.conditions[beggining_of_column_name + 'Christian?'][country]

        if idx == 0:
            return MULTIPLIER_RELIGIOUS_DENOMINATION_1ST * value_for_agents_religious_denomination
        elif idx == 1:
            return MULTIPLIER_RELIGIOUS_DENOMINATION_2ND * value_for_agents_religious_denomination
        elif idx == 2:
            return MULTIPLIER_RELIGIOUS_DENOMINATION_3RD * value_for_agents_religious_denomination

    def _calculate_happiness_based_on_a_sense_of_comfort_with_immigrants(self, country):
        if self.conditions['Feel comfortable with all the social relations'][country] >= \
                self.conditions['Feel uncomfortable with at least one social relation'][country]:
            return MULTIPLIER_SENSE_OF_COMFORT * \
                    self.conditions['Feel comfortable with all the social relations'][country]
        else:
            return -MULTIPLIER_SENSE_OF_COMFORT * \
                   self.conditions['Feel uncomfortable with at least one social relation'][country]

    def _calculate_happiness_based_on_integration_frequency_with_immigrants(self, country):
        if self.conditions['Frequent integration'][country] >= self.conditions['Less frequent integration'][country]:
            return MULTIPLIER_INTEGRATION_WITH_IMMIGRANTS * self.conditions['Frequent integration'][country]
        else:
            return -MULTIPLIER_INTEGRATION_WITH_IMMIGRANTS * self.conditions['Less frequent integration'][country]

    def _calculate_happiness_based_on_thoughts_about_migration_into_own_country(self, country):
        agree = self.conditions['Strongly agree'][country] + self.conditions['Agree'][country]
        disagree = self.conditions['Strongly disagree'][country] + self.conditions['Disagree'][country]
        if agree >= disagree:
            return 2 * MULTIPLIER_MIGRATION_INTO_OUR_COUNTRY * self.conditions['Strongly agree'][
                country] + MULTIPLIER_MIGRATION_INTO_OUR_COUNTRY * self.conditions['Agree'][country]
        else:
            return -(2 * MULTIPLIER_MIGRATION_INTO_OUR_COUNTRY * self.conditions['Strongly disagree'][
                country] + MULTIPLIER_MIGRATION_INTO_OUR_COUNTRY * self.conditions['Disagree'][country])


In [5]:
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation


class EUMap(Model):
    def __init__(self, agents_attribute, conditions, agents_count=100):

        self.schedule = RandomActivation(self)
        probability_country = agents_attribute['Population'] / agents_attribute['Population'].sum()
        countries_for_agents = np.random.choice(agents_attribute.index, agents_count, p=probability_country.values)
        for i, origin_country in enumerate(countries_for_agents):
            is_immigrant = np.random.choice([True, False], 1,
                                            p=[agents_attribute.loc[origin_country]['Immigrants probability'],
                                               1 - agents_attribute.loc[origin_country]['Immigrants probability']])[0]
            if is_immigrant:
                current_country = origin_country
                while current_country == origin_country:
                    current_country = np.random.choice(agents_attribute.index, 1, p=probability_country.values)[0]
            else:
                current_country = origin_country
            likely_to_travel = np.random.choice([True, False], 1,
                                                p=[agents_attribute.loc[origin_country]['Totally likely'],
                                                   agents_attribute.loc[origin_country]['Not likely']])[0]
            race = np.random.choice(['White', 'Black', 'Asian'], 1,
                                    p=[agents_attribute.loc[origin_country]['White people'],
                                       agents_attribute.loc[origin_country]['Black people'],
                                       agents_attribute.loc[origin_country]['Asian people']])[0]
            religious_denomination = np.random.choice(['Jew', 'Muslim', 'Christian'], 1,
                                                      p=[agents_attribute.loc[origin_country]['Jews'],
                                                         agents_attribute.loc[origin_country]['Muslims'],
                                                         agents_attribute.loc[origin_country]['Christians']])[0]
            agent = Person(self, i, origin_country, current_country, race, religious_denomination, likely_to_travel,
                           conditions)
            self.schedule.add(agent)

        self.current_country_collector = DataCollector(
            agent_reporters={"current_country": "current_country"})

    def step(self):
        self.current_country_collector.collect(self)
        self.schedule.step()

### Data loading
Data required for migration simulation (prepared in *data_analysis.ipynb*)

In [6]:
import pandas as pd


def read_data_from_csv(filename: str) -> pd.DataFrame:
    return pd.read_csv('data/' + filename + '.csv', sep=';', index_col='Country')


df_agents_attribute = read_data_from_csv('agents_attributes')
df_conditions = read_data_from_csv('conditions')

### Migration simulation

In [7]:
n_iter = 25
eu_map = EUMap(df_agents_attribute, df_conditions)
for s in range(n_iter):
    print('Step: {}'.format(s))
    eu_map.step()

Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24


### Analysis of gathered data

In all steps current country for each agent is saved for future analysis.

In [8]:
agent_current_countries = eu_map.current_country_collector.get_agent_vars_dataframe()
df_result_visited_countries = pd.DataFrame.from_records(
    [agent['current_country'].tolist() for _, agent in agent_current_countries.reset_index().groupby(['AgentID'])])
display(df_result_visited_countries)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,France,Luxemburg,Germany,Luxemburg,France,Netherlands,United Kingdom,Ireland,United Kingdom,France,...,Netherlands,Germany,United Kingdom,Denmark,Germany,Poland,Sweden,Estonia,Sweden,Germany
1,United Kingdom,Germany,Luxemburg,France,Germany,United Kingdom,Germany,France,Italy,Austria,...,Netherlands,France,Netherlands,Belgium,Luxemburg,France,United Kingdom,Ireland,Belgium,Ireland
2,France,United Kingdom,Germany,Netherlands,Ireland,Belgium,Germany,Denmark,Sweden,Lithuania,...,Germany,Netherlands,France,Spain,Portugal,Spain,France,Germany,France,Belgium
3,Spain,France,Netherlands,United Kingdom,Belgium,Netherlands,Germany,Belgium,United Kingdom,Netherlands,...,Denmark,United Kingdom,Denmark,United Kingdom,Belgium,Netherlands,Germany,United Kingdom,Germany,Poland
4,Portugal,Spain,France,Belgium,Luxemburg,Germany,France,Italy,Austria,Czech Republic,...,Netherlands,United Kingdom,France,Italy,Austria,Italy,Greece,Bulgaria,Romania,Hungary
5,France,Luxemburg,Germany,Austria,Czech Republic,Austria,Germany,Netherlands,Germany,Belgium,...,Luxemburg,Germany,United Kingdom,France,Luxemburg,Germany,United Kingdom,Ireland,United Kingdom,France
6,Denmark,Germany,France,United Kingdom,Ireland,Netherlands,France,Spain,Portugal,Spain,...,Germany,France,Italy,Greece,Cyprus,Greece,Cyprus,Greece,Bulgaria,Romania
7,Poland,Germany,Netherlands,Ireland,Belgium,Ireland,United Kingdom,France,Spain,France,...,Germany,Belgium,France,Germany,Czech Republic,Poland,Czech Republic,Germany,Sweden,Latvia
8,Germany,Poland,Germany,Belgium,Netherlands,Ireland,Belgium,Germany,Austria,Slovenia,...,Cyprus,Greece,Cyprus,Greece,Italy,Greece,Italy,Slovenia,Austria,Slovenia
9,Germany,Denmark,Germany,Belgium,Ireland,Netherlands,Belgium,France,Italy,Slovenia,...,Croatia,Slovenia,Hungary,Austria,Hungary,Austria,Italy,France,Luxemburg,France
